In [1]:
import pandas as pd
import matplotlib as plt
import scipy.stats as stats
import numpy as np

In [2]:
df = pd.read_csv('data/weatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(subset = ['RainTomorrow'], inplace = True)

In [ ]:
df.drop(columns = 'Evaporation', inplace=True)

In [3]:
df_nums = df.select_dtypes('float64')
df_nums.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7


In [9]:
df_nums.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [4]:
df2 = df_nums.copy()

In [ ]:
df2[(np.abs(stats.zscore(df2)) < 3).all(axis=1)]

In [5]:
df_test = df2.drop(columns=['Cloud9am', 'Cloud3pm'])
df_test.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,NaN,NaN,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8
1,7.4,25.1,0.0,NaN,NaN,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3
2,12.9,25.7,0.0,NaN,NaN,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2
3,9.2,28.0,0.0,NaN,NaN,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5
4,17.5,32.3,1.0,NaN,NaN,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7


In [ ]:
features = df_test.columns
for feature in features:
    qupper = df_test[feature].quantile(0.99)
    print(feature, len(df_test.loc[df_test[feature] < qupper]))

In [6]:
features = df_test.columns
for feature in features:
    qupper = df_test[feature].quantile(0.99)
    df_test = df_test.loc[(df_test[feature].isna()) | (df_test[feature] < qupper)]

In [7]:
df_test.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
count,124288.000000,124500.000000,122725.000000,70956.000000,65093.000000,116324.000000,124013.000000,122818.000000,123287.000000,121741.000000,112096.000000,112124.000000,124061.000000,122534.00000
mean,11.879099,22.890806,1.690949,5.023269,7.583706,38.806454,13.429931,18.122018,69.060485,51.370656,1017.901969,1015.503054,16.678445,21.39659
std,6.062426,6.396897,4.643579,3.106468,3.700301,11.818931,7.980985,7.948135,17.574703,19.360066,6.525393,6.414607,5.938739,6.23113
min,-8.500000,-3.800000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000,1.000000,1.000000,982.200000,978.200000,-6.200000,-5.10000
25%,7.600000,18.100000,0.000000,2.600000,4.900000,31.000000,7.000000,13.000000,58.000000,37.000000,1013.600000,1011.200000,12.400000,16.80000
50%,11.800000,22.500000,0.000000,4.600000,8.400000,37.000000,13.000000,17.000000,70.000000,52.000000,1018.000000,1015.600000,16.600000,21.10000
75%,16.300000,27.600000,0.600000,7.000000,10.600000,46.000000,19.000000,24.000000,82.000000,65.000000,1022.500000,1020.100000,21.000000,25.90000
max,25.700000,39.700000,37.900000,17.200000,13.300000,78.000000,35.000000,39.000000,99.000000,96.000000,1033.900000,1029.800000,29.800000,35.90000


In [8]:
df_test.isna().sum()

MinTemp           1361
MaxTemp           1149
Rainfall          2924
Evaporation      54693
Sunshine         60556
WindGustSpeed     9325
WindSpeed9am      1636
WindSpeed3pm      2831
Humidity9am       2362
Humidity3pm       3908
Pressure9am      13553
Pressure3pm      13525
Temp9am           1588
Temp3pm           3115
dtype: int64

In [10]:
df_test.drop(columns =['Evaporation', 'Sunshine'], axis = 1, inplace=True)

In [19]:
df_test.describe()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
count,124288.000000,124500.000000,122725.000000,116324.000000,124013.000000,122818.000000,123287.000000,121741.000000,112096.000000,112124.000000,124061.000000,122534.00000
mean,11.879099,22.890806,1.690949,38.806454,13.429931,18.122018,69.060485,51.370656,1017.901969,1015.503054,16.678445,21.39659
std,6.062426,6.396897,4.643579,11.818931,7.980985,7.948135,17.574703,19.360066,6.525393,6.414607,5.938739,6.23113
min,-8.500000,-3.800000,0.000000,7.000000,0.000000,0.000000,1.000000,1.000000,982.200000,978.200000,-6.200000,-5.10000
25%,7.600000,18.100000,0.000000,31.000000,7.000000,13.000000,58.000000,37.000000,1013.600000,1011.200000,12.400000,16.80000
50%,11.800000,22.500000,0.000000,37.000000,13.000000,17.000000,70.000000,52.000000,1018.000000,1015.600000,16.600000,21.10000
75%,16.300000,27.600000,0.600000,46.000000,19.000000,24.000000,82.000000,65.000000,1022.500000,1020.100000,21.000000,25.90000
max,25.700000,39.700000,37.900000,78.000000,35.000000,39.000000,99.000000,96.000000,1033.900000,1029.800000,29.800000,35.90000


In [14]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3, weights="uniform")
imputed = imputer.fit_transform(df_test)
df_imputed = pd.DataFrame(imputed, index = df_test.index)

In [18]:
df_imputed.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000,125649.000000
mean,11.875725,22.891085,1.689245,38.586038,13.422087,18.085476,69.090784,51.441735,1018.024645,1015.693350,16.662932,21.422356
std,6.040965,6.378827,4.608092,11.569430,7.952220,7.915570,17.489899,19.178445,6.313065,6.226313,5.932043,6.194495
min,-8.500000,-3.800000,0.000000,7.000000,0.000000,0.000000,1.000000,1.000000,982.200000,978.200000,-6.200000,-5.100000
25%,7.600000,18.200000,0.000000,30.000000,7.000000,13.000000,58.000000,38.000000,1014.000000,1011.600000,12.400000,16.900000
50%,11.800000,22.600000,0.000000,37.000000,13.000000,17.000000,70.000000,52.000000,1018.166667,1015.800000,16.600000,21.200000
75%,16.300000,27.600000,0.600000,46.000000,19.000000,24.000000,82.000000,64.666667,1022.400000,1020.066667,21.000000,25.800000
max,25.700000,39.700000,37.900000,78.000000,35.000000,39.000000,99.000000,96.000000,1033.900000,1029.800000,29.800000,35.900000


In [16]:
df_imputed.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
dtype: int64

In [17]:
df_imputed

,0,1,2,3,4,5,6,7,8,9,10,11
0,13.4,22.9,0.6,44.000000,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8
1,7.4,25.1,0.0,44.000000,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3
2,12.9,25.7,0.0,46.000000,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2
3,9.2,28.0,0.0,24.000000,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5
4,17.5,32.3,1.0,41.000000,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7
...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.4,0.0,31.000000,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4
145456,3.6,25.3,0.0,22.000000,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5
145457,5.4,26.9,0.0,37.000000,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1
145458,7.8,27.0,0.0,28.000000,13.0,7.0,51.0,24.0,1019.4,1016.5,15.1,26.0


In [ ]:
df.Humidity9am.plot.hist()

In [ ]:
df.MaxTemp.plot.hist()

In [ ]:
df.Rainfall.plot.hist()

In [ ]:
# MinTemp - impute mean
# Max Temp - impute mean
# 